# Notebook for developing code to go into structural_attack class

In [1]:
import sys
import os


#for development use local copy of aisdc in preference to installed version
sys.path.insert(0,os.path.abspath(".."))


In [2]:
import json

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC


from aisdc.attacks.structural_attack import StructuralAttack  # pylint: disable = import-error
from aisdc.attacks.target import Target  # pylint: disable = import-error

In [3]:
# [Researcher] Access a dataset
X, y = load_breast_cancer(return_X_y=True, as_frame=False)

# [Researcher] Split into training and test sets
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

In [19]:
# [Researcher] Define the classifier
def get_target(modeltype:str)->Target:
    #these should be handled
    if modeltype =='dt':
        target_model = DecisionTreeClassifier()
    elif modeltype =='rf':
        target_model = RandomForestClassifier()
    elif modeltype =='xgb':
        target_model = XGBClassifier()
    # should get polite error but not DoF yet    
    elif modeltype == 'svc' :
        target_model = SVC()
    else:
        raise NotImplementedError('model type passed to get_model unknown')
        
    # [Researcher] Train the classifier
    target_model.fit(train_X, train_y)

    # [Researcher] Provide the model and the train and test data to the TRE

    # [TRE] Compute the predictions on the training and test sets
    #train_preds = target_model.predict_proba(train_X)
    #test_preds = target_model.predict_proba(test_X)

    # [TRE / Researcher] Wrap the model and data in a Target object
    target = Target(
        model=target_model)
    target.add_processed_data(train_X, train_y, test_X, test_y)
    
    return target

In [20]:
import importlib
import aisdc.attacks.structural_attack
importlib.reload(aisdc.attacks.structural_attack)
from aisdc.attacks.structural_attack import StructuralAttack

In [22]:
tree_types = ['dt','rf','xgb','svc','other']

for mytype in tree_types:
    print( f'====={mytype} ======')
    target = get_target(mytype)
    myattack = StructuralAttack()
    ret= myattack.attack(target)

    print(ret)

=====dt ======
could do more, 
model type is DecisionTreeClassifier
 criterion: gini 
 splitter: best 
 max_depth: None 
 min_samples_split: 2 
 min_samples_leaf: 1 
 min_weight_fraction_leaf: 0.0 
 max_features: None 
 max_leaf_nodes: None 
 random_state: None 
 min_impurity_decrease: 0.0 
 class_weight: None 
 ccp_alpha: 0.0 
 n_features_in_: 30 
 n_outputs_: 1 
 classes_: [0 1] 
 n_classes_: 2 
 max_features_: 30 
 tree_: <sklearn.tree._tree.Tree object at 0x283a90180> 

=====rf ======
could do more, 
model type is RandomForestClassifier
 base_estimator: DecisionTreeClassifier() 
 n_estimators: 100 
 estimator_params: ('criterion', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'random_state', 'ccp_alpha') 
 bootstrap: True 
 oob_score: False 
 n_jobs: None 
 random_state: None 
 verbose: 0 
 warm_start: False 
 class_weight: None 
 max_samples: None 
 criterion: gini 
 max_depth: None 
 mi

NotImplementedError: model type passed to get_model unknown